## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import lyricsgenius
from matplotlib import pyplot as plt
import json
import pickle

#For MSE models
import nltk
from nltk.util import bigrams, trigrams
from nltk.util import pad_sequence
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize.treebank import TreebankWordTokenizer, TreebankWordDetokenizer
from nltk.lm import MLE


#For RNN model
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization,LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf

Using TensorFlow backend.


## Importing songs

In [2]:
"""
genius = lyricsgenius.Genius("jOoA2dCGT_F8pHJJmmd3HgXotfZlbpDr2Dwd9QTA2v122saEfQ3S-RDCu_xd76u7")

genius.skip_non_songs = True
genius.verbose = False
genius.remove_section_headers = True
genius.excluded_terms = ["(Remix)", "(Live)"]

rapper = 'Oxxxymiron'
num_songs = 50

artist = genius.search_artist(rapper, max_songs=num_songs)
print(artist.songs)

lyrics = []
for i in range(len(artist.songs)):
    song = [artist.songs[i].lyrics]
    lyrics.append(song)
    
with open("oxxxymiron_lyrics.txt", "wb") as fp:
    pickle.dump(lyrics, fp)

"""

'\ngenius = lyricsgenius.Genius("jOoA2dCGT_F8pHJJmmd3HgXotfZlbpDr2Dwd9QTA2v122saEfQ3S-RDCu_xd76u7")\n\ngenius.skip_non_songs = True\ngenius.verbose = False\ngenius.remove_section_headers = True\ngenius.excluded_terms = ["(Remix)", "(Live)"]\n\nrapper = \'Oxxxymiron\'\nnum_songs = 50\n\nartist = genius.search_artist(rapper, max_songs=num_songs)\nprint(artist.songs)\n\nlyrics = []\nfor i in range(len(artist.songs)):\n    song = [artist.songs[i].lyrics]\n    lyrics.append(song)\n    \nwith open("oxxxymiron_lyrics.txt", "wb") as fp:\n    pickle.dump(lyrics, fp)\n\n'

In [3]:
#Unpickling our songs
with open("oxxxymiron_lyrics.txt", "rb") as fp:   
    lyrics = pickle.load(fp)

In [4]:
print(len(lyrics))

50


In [7]:
print(lyrics[:2])

[["Дон ли, Волга ли течёт — котомку на плечо\nБоль в груди — там тайничок, открытый фомкой, не ключом\nСколько миль ещё? Перелет короткий был не в счёт\nДолгий пыльный чёс, фургон набит коробками с мерчём\nВерим — подфартит, наши постели портативны\nМенестрелю два пути: корпоратив или квартирник\nСхемы однотипны, все теперь MC\nВедь, смену породив, мы здесь достигли смены парадигмы\nТеперь рэп — многопартийный; бэттлов наплодив\nЯ смотрю в зеркало по типу: «Сколько бед наворотил ты!»\nЯ б весь рэп поработил, но всё время в пути\nУ индустрии нервный тик, валокордин — стенокардийным\nСоберите суд, но победителей не судят\nМы первые кроманьонцы — мы выбились в люди\nНе пизди! Я кладу на вас, челядь, пятикратно\nВедь мы выступаем сильно, будто челюсть питекантропа\n\nВесь мой рэп, если коротко, про то, что\nУж который год который город под подошвой\nВ гору, когда прёт; потом под гору, когда тошно\nЯ не то, что Гулливер, но всё же город под подошвой\nГород под подошвой, город под подошвой\n

## Preparing our set of sentences

In [6]:
padded_songs = list(flatten(pad_both_ends(song, n=2) for song in lyrics))

In [7]:
print(padded_songs[:3])

['<s>', "Дон ли, Волга ли течёт — котомку на плечо\nБоль в груди — там тайничок, открытый фомкой, не ключом\nСколько миль ещё? Перелет короткий был не в счёт\nДолгий пыльный чёс, фургон набит коробками с мерчём\nВерим — подфартит, наши постели портативны\nМенестрелю два пути: корпоратив или квартирник\nСхемы однотипны, все теперь MC\nВедь, смену породив, мы здесь достигли смены парадигмы\nТеперь рэп — многопартийный; бэттлов наплодив\nЯ смотрю в зеркало по типу: «Сколько бед наворотил ты!»\nЯ б весь рэп поработил, но всё время в пути\nУ индустрии нервный тик, валокордин — стенокардийным\nСоберите суд, но победителей не судят\nМы первые кроманьонцы — мы выбились в люди\nНе пизди! Я кладу на вас, челядь, пятикратно\nВедь мы выступаем сильно, будто челюсть питекантропа\n\nВесь мой рэп, если коротко, про то, что\nУж который год который город под подошвой\nВ гору, когда прёт; потом под гору, когда тошно\nЯ не то, что Гулливер, но всё же город под подошвой\nГород под подошвой, город под подо

In [8]:
#Standardize our text
bad_chars = [';', ':', '!', '*', 'n', ',', '.', '"', "'", '(', ')', '?', '—', '\\', '[', ']', '»', '«']
for i in padded_songs:
    for char in bad_chars:
        padded_songs = str(padded_songs).replace(char,' ').lower()

In [9]:
print(padded_songs)

  <s>    дон ли  волга ли течёт   котомку на плечо  боль в груди   там тайничок  открытый фомкой  не ключом  сколько миль ещё  перелет короткий был не в счёт  долгий пыльный чёс  фургон набит коробками с мерчём  верим   подфартит  наши постели портативны  менестрелю два пути  корпоратив или квартирник  схемы однотипны  все теперь mc  ведь  смену породив  мы здесь достигли смены парадигмы  теперь рэп   многопартийный  бэттлов наплодив  я смотрю в зеркало по типу   сколько бед наворотил ты    я б весь рэп поработил  но всё время в пути  у индустрии нервный тик  валокордин   стенокардийным  соберите суд  но победителей не судят  мы первые кроманьонцы   мы выбились в люди  не пизди  я кладу на вас  челядь  пятикратно  ведь мы выступаем сильно  будто челюсть питекантропа    весь мой рэп  если коротко  про то  что  уж который год который город под подошвой  в гору  когда прёт  потом под гору  когда тошно  я не то  что гулливер  но всё же город под подошвой  город под подошвой  город под подо

In [10]:
#Break our text into separate sentences
sentences = nltk.tokenize.sent_tokenize(padded_songs)

In [11]:
#Split our sentences into separate words
words = []
for i in sentences:
    words.append(i.split())
words = list(flatten(words))

In [12]:
print(words[:30])

['<s>', 'дон', 'ли', 'волга', 'ли', 'течёт', 'котомку', 'на', 'плечо', 'боль', 'в', 'груди', 'там', 'тайничок', 'открытый', 'фомкой', 'не', 'ключом', 'сколько', 'миль', 'ещё', 'перелет', 'короткий', 'был', 'не', 'в', 'счёт', 'долгий', 'пыльный', 'чёс']


## Preparing ngrams

In [13]:
#Break into bigrams
bigrams = list(bigrams(words))

In [14]:
print(bigrams[:5])

[('<s>', 'дон'), ('дон', 'ли'), ('ли', 'волга'), ('волга', 'ли'), ('ли', 'течёт')]


In [15]:
#Break into trigrams
trigrams = list(trigrams(words))

In [16]:
print(trigrams[:5])

[('<s>', 'дон', 'ли'), ('дон', 'ли', 'волга'), ('ли', 'волга', 'ли'), ('волга', 'ли', 'течёт'), ('ли', 'течёт', 'котомку')]


#### Separating train / test ngrams

In [17]:
bi_train_size = int(len(bigrams) * 0.8)
bi_test_size = int(len(bigrams) * 0.2)

train_bigrams = bigrams[:bi_train_size]
test_bigrams = bigrams[bi_test_size:]

In [18]:
tri_train_size = int(len(trigrams) * 0.8)
tri_test_size = int(len(trigrams) * 0.2)

train_trigrams = bigrams[:tri_train_size]
test_trigrams = bigrams[tri_test_size:]

In [19]:
#Make bigram counts and vocabulary
bi_train, bi_vocab = padded_everygram_pipeline(2, train_bigrams)

In [20]:
#Make trigram counts and vocabulary
tri_train, tri_vocab = padded_everygram_pipeline(3, train_trigrams)

## Maximum Likelihood Estimator (MLE) models

In [21]:
#Make a function to detokenize our final text
detok = TreebankWordDetokenizer().detokenize

In [22]:
#Make a function to generate a random text and clear it from <s>...</s>
def generator(model, words, random_seed=20):
    rap = []
    for word in model.generate(words, random_seed=random_seed):
        if word == "<s>" or word == "</s>":
            continue
        rap.append(word)
    return detok(rap)

#### Bigram model

In [23]:
#Set Max Likelihood Estimator model for bigrams
bi_lm = MLE(2)

In [24]:
#Train the bigram model
bi_lm.fit(bi_train, bi_vocab)

In [25]:
#Generate a text
generator(bi_lm, 50, random_seed=40)

'ты пиросмани конечно братства вес в вашем доме быть buff или сохранять тебя вы лев и так но город беден ты'

As a result, the bigram MLE model generated the verse, which perfectly reflects the quality of this model.

Unfortunately, I did find the way to stop predicting padding sings, that is why the number of predicted words sometimes is different from the one that was indicated. Changing padding from verses to songs did not help much.

Nevertheless, the model does its work by finding neigbouring words throug the bigrams.
Probably, the model performance could be improved by better processing of input data and larger dataset.

In [26]:
#Calculating model entropy
bi_lm.entropy(test_bigrams)

inf

In [27]:
#Calculating model perplexity
bi_lm.perplexity(test_bigrams)

inf

Unfortunatelly, model entropy and perplexity for some reason tend to infinity on the test bigrams. 

If not because of some code or math problem, probably the model encountered such pairs of words (or even words) that has never seen before. That is why, it fails to generalize this data. 

In [28]:
bi_lm.entropy(train_bigrams)

3.6913928053562075

In [29]:
bi_lm.perplexity(train_bigrams)

12.918734121883936

However, using the data on which the model was trained, we receive more realistic result (not suprising). Though, it is may be incorrent to use training data, now we may guess that the problem can be in input data quality.

#### Trigram model

In [30]:
#Set Max Likelihood Estimator model for trigrams
tri_lm = MLE(3)

In [31]:
#Train the trigram model
tri_lm.fit(tri_train, tri_vocab)

In [32]:
#Generate a text
generator(tri_lm, 20, random_seed=35)

'мне подождать'

In [33]:
tri_lm.entropy(test_trigrams)

inf

In [34]:
tri_lm.perplexity(test_trigrams)

inf

In [35]:
tri_lm.entropy(train_trigrams)

3.7110383225123598

In [36]:
tri_lm.perplexity(train_trigrams)

13.09585476787456

In trigram model we have quite the same problem, though entropy and perplexity are a bit higher on training sample. If it was test sample, we might conclude that this model is worse on text generalization.

## RNN model

Further on, I decided to construct RNN LSTM model, which will give us some hot rapping.

In [37]:
clear_lyrics = lyrics

for i in clear_lyrics:
    for char in bad_chars:
        clear_lyrics = str(clear_lyrics).replace(char,' ').lower()

In [38]:
#Make a function to tokenize & filter input data
def tokenize_words(input): 
    #Initiate the tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)
    
    #Filter some words if they are in nltk stop list
    filtered = filter(lambda token: token not in stopwords.words('russian'), tokens)
    return " ".join(filtered)

In [39]:
#Preprocess our input data and make tokens
processed_inputs = tokenize_words(clear_lyrics)

In [40]:
print(processed_inputs[:300])

дон волга течёт котомку плечо боль груди тайничок открытый фомкой ключом сколько миль ещё перелет короткий счёт долгий пыльный чёс фургон набит коробками мерчём верим подфартит наши постели портативны менестрелю пути корпоратив квартирник схемы однотипны mc смену породив достигли смены парадигмы рэп


In [41]:
#Since NN requires numbers, but not text characters, convert the characters in our input to numbers
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [42]:
#For futher data preparation, calculate the length of our input and vocabulary
input_len = len(processed_inputs)
vocab_len = len(chars)

print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 89852
Total vocab: 74


In [43]:
#Define how long an individual sequence (one complete mapping of inputs characters as integers) to be
seq_length = 100

#Make empty lists to store our input and output data
x_data = []
y_data = []

In [44]:
#Loop through inputs - start at the beginning and go until we hit the final character we can create a sequence out of
for i in range(0, input_len - seq_length, 1):
    
    #Define input and output sequences
    #Input is the current character plus desired sequence length
    in_seq = processed_inputs[i:i + seq_length]

    #Output sequence is the initial character plus total sequence length
    out_seq = processed_inputs[i + seq_length]

    #Convert list of characters to integers and add the values to our lists
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

In [45]:
#Total number of sequences
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 89752


In [46]:
#Transform it to numpy array for our NN
X = np.reshape(x_data, (n_patterns, seq_length, 1))

#Make the numbers float for activation function being able to interpret them as probabilities
X = X/float(vocab_len)

In [47]:
#One-hot encode our data
y = np_utils.to_categorical(y_data)

In [48]:
#Construct stacked LSTM model
model = Sequential()

model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(256, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(128))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(y.shape[1], activation='softmax'))

In [49]:
#For our callback create a file with RNN weights
filename = "RNN_weights.hdf5"
filepath = "RNN_weights.hdf5"

In [50]:
#Set callback function to save the best weights
callbacks = [ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')]

In [51]:
#Compile RNN model on default adam optimizer and track categorical cross-entropy
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [52]:
history = model.fit(X, y, epochs=700, batch_size=256, callbacks=callbacks)

Epoch 1/700
89752/89752 [==============================] - 116s 1ms/step - loss: 3.3559

Epoch 00001: loss improved from inf to 3.35589, saving model to RNN_weights.hdf5
Epoch 2/700
89752/89752 [==============================] - 114s 1ms/step - loss: 2.9859

Epoch 00002: loss improved from 3.35589 to 2.98593, saving model to RNN_weights.hdf5
Epoch 3/700
89752/89752 [==============================] - 114s 1ms/step - loss: 2.9063

Epoch 00003: loss improved from 2.98593 to 2.90632, saving model to RNN_weights.hdf5
Epoch 4/700
89752/89752 [==============================] - 114s 1ms/step - loss: 2.8594

Epoch 00004: loss improved from 2.90632 to 2.85941, saving model to RNN_weights.hdf5
Epoch 5/700
89752/89752 [==============================] - 115s 1ms/step - loss: 2.8246

Epoch 00005: loss improved from 2.85941 to 2.82461, saving model to RNN_weights.hdf5
Epoch 6/700
89752/89752 [==============================] - 114s 1ms/step - loss: 2.7810

Epoch 00006: loss improved from 2.82461 to 2.

89752/89752 [==============================] - 113s 1ms/step - loss: 1.5639

Epoch 00096: loss improved from 1.56647 to 1.56394, saving model to RNN_weights.hdf5
Epoch 97/700
89752/89752 [==============================] - 113s 1ms/step - loss: 1.5622

Epoch 00097: loss improved from 1.56394 to 1.56218, saving model to RNN_weights.hdf5
Epoch 98/700
89752/89752 [==============================] - 113s 1ms/step - loss: 1.5517

Epoch 00098: loss improved from 1.56218 to 1.55169, saving model to RNN_weights.hdf5
Epoch 99/700
89752/89752 [==============================] - 113s 1ms/step - loss: 1.5397

Epoch 00099: loss improved from 1.55169 to 1.53968, saving model to RNN_weights.hdf5
Epoch 100/700
89752/89752 [==============================] - 114s 1ms/step - loss: 1.5400

Epoch 00100: loss did not improve from 1.53968
Epoch 101/700
89752/89752 [==============================] - 113s 1ms/step - loss: 1.5329

Epoch 00101: loss improved from 1.53968 to 1.53292, saving model to RNN_weights.hdf5

89752/89752 [==============================] - 115s 1ms/step - loss: 1.3415

Epoch 00144: loss did not improve from 1.34062
Epoch 145/700
89752/89752 [==============================] - 113s 1ms/step - loss: 1.3355

Epoch 00145: loss improved from 1.34062 to 1.33548, saving model to RNN_weights.hdf5
Epoch 146/700
89752/89752 [==============================] - 113s 1ms/step - loss: 1.3284

Epoch 00146: loss improved from 1.33548 to 1.32838, saving model to RNN_weights.hdf5
Epoch 147/700
89752/89752 [==============================] - 113s 1ms/step - loss: 1.3320

Epoch 00147: loss did not improve from 1.32838
Epoch 148/700
89752/89752 [==============================] - 113s 1ms/step - loss: 1.3237

Epoch 00148: loss improved from 1.32838 to 1.32367, saving model to RNN_weights.hdf5
Epoch 149/700
89752/89752 [==============================] - 113s 1ms/step - loss: 1.3220

Epoch 00149: loss improved from 1.32367 to 1.32199, saving model to RNN_weights.hdf5
Epoch 150/700
89752/89752 [=======

89752/89752 [==============================] - 113s 1ms/step - loss: 1.0949

Epoch 00245: loss did not improve from 1.08955
Epoch 246/700
89752/89752 [==============================] - 113s 1ms/step - loss: 1.0893

Epoch 00246: loss improved from 1.08955 to 1.08926, saving model to RNN_weights.hdf5
Epoch 247/700
89752/89752 [==============================] - 113s 1ms/step - loss: 1.0904

Epoch 00247: loss did not improve from 1.08926
Epoch 248/700
89752/89752 [==============================] - 114s 1ms/step - loss: 1.0828

Epoch 00248: loss improved from 1.08926 to 1.08282, saving model to RNN_weights.hdf5
Epoch 249/700
89752/89752 [==============================] - 115s 1ms/step - loss: 1.0829

Epoch 00249: loss did not improve from 1.08282
Epoch 250/700
89752/89752 [==============================] - 114s 1ms/step - loss: 1.0815

Epoch 00250: loss improved from 1.08282 to 1.08145, saving model to RNN_weights.hdf5
Epoch 251/700
89752/89752 [==============================] - 114s 1ms/st

89752/89752 [==============================] - 114s 1ms/step - loss: 1.0193

Epoch 00299: loss did not improve from 1.01814
Epoch 300/700
89752/89752 [==============================] - 113s 1ms/step - loss: 1.0162

Epoch 00300: loss improved from 1.01814 to 1.01624, saving model to RNN_weights.hdf5
Epoch 301/700
89752/89752 [==============================] - 114s 1ms/step - loss: 1.0089

Epoch 00301: loss improved from 1.01624 to 1.00885, saving model to RNN_weights.hdf5
Epoch 302/700
89752/89752 [==============================] - 114s 1ms/step - loss: 1.0046

Epoch 00302: loss improved from 1.00885 to 1.00462, saving model to RNN_weights.hdf5
Epoch 303/700
89752/89752 [==============================] - 114s 1ms/step - loss: 1.0181

Epoch 00303: loss did not improve from 1.00462
Epoch 304/700
89752/89752 [==============================] - 114s 1ms/step - loss: 1.0133

Epoch 00304: loss did not improve from 1.00462
Epoch 305/700
89752/89752 [==============================] - 114s 1ms/st

89752/89752 [==============================] - 114s 1ms/step - loss: 0.9658

Epoch 00354: loss did not improve from 0.95614
Epoch 355/700
89752/89752 [==============================] - 114s 1ms/step - loss: 0.9598

Epoch 00355: loss did not improve from 0.95614
Epoch 356/700
89752/89752 [==============================] - 114s 1ms/step - loss: 0.9583

Epoch 00356: loss did not improve from 0.95614
Epoch 357/700
89752/89752 [==============================] - 114s 1ms/step - loss: 0.9552

Epoch 00357: loss improved from 0.95614 to 0.95518, saving model to RNN_weights.hdf5
Epoch 358/700
89752/89752 [==============================] - 114s 1ms/step - loss: 0.9545

Epoch 00358: loss improved from 0.95518 to 0.95453, saving model to RNN_weights.hdf5
Epoch 359/700
89752/89752 [==============================] - 114s 1ms/step - loss: 0.9501

Epoch 00359: loss improved from 0.95453 to 0.95014, saving model to RNN_weights.hdf5
Epoch 360/700
89752/89752 [==============================] - 114s 1ms/st

89752/89752 [==============================] - 135s 2ms/step - loss: 0.9187

Epoch 00410: loss did not improve from 0.91541
Epoch 411/700
89752/89752 [==============================] - 137s 2ms/step - loss: 0.9207

Epoch 00411: loss did not improve from 0.91541
Epoch 412/700
89752/89752 [==============================] - 180s 2ms/step - loss: 0.9110

Epoch 00412: loss improved from 0.91541 to 0.91101, saving model to RNN_weights.hdf5
Epoch 413/700
89752/89752 [==============================] - 215s 2ms/step - loss: 0.9177

Epoch 00413: loss did not improve from 0.91101
Epoch 414/700
89752/89752 [==============================] - 187s 2ms/step - loss: 0.9192

Epoch 00414: loss did not improve from 0.91101
Epoch 415/700
89752/89752 [==============================] - 203s 2ms/step - loss: 0.9109

Epoch 00415: loss improved from 0.91101 to 0.91094, saving model to RNN_weights.hdf5
Epoch 416/700
89752/89752 [==============================] - 212s 2ms/step - loss: 0.9096

Epoch 00416: loss i

89752/89752 [==============================] - 124s 1ms/step - loss: 0.8781

Epoch 00465: loss did not improve from 0.87669
Epoch 466/700
89752/89752 [==============================] - 125s 1ms/step - loss: 0.8810

Epoch 00466: loss did not improve from 0.87669
Epoch 467/700
89752/89752 [==============================] - 124s 1ms/step - loss: 0.8905

Epoch 00467: loss did not improve from 0.87669
Epoch 468/700
89752/89752 [==============================] - 125s 1ms/step - loss: 0.8787

Epoch 00468: loss did not improve from 0.87669
Epoch 469/700
89752/89752 [==============================] - 124s 1ms/step - loss: 0.8868

Epoch 00469: loss did not improve from 0.87669
Epoch 470/700
89752/89752 [==============================] - 126s 1ms/step - loss: 0.8792

Epoch 00470: loss did not improve from 0.87669
Epoch 471/700
89752/89752 [==============================] - 123s 1ms/step - loss: 0.8825

Epoch 00471: loss did not improve from 0.87669
Epoch 472/700
89752/89752 [=====================

KeyboardInterrupt: 

In [53]:
#Loading pre-trained weights
model.load_weights(filename)

In [54]:
# Plot loss values
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

NameError: name 'history' is not defined

On the graph, we can see the gradual decrease of cross-entropy loss, which is a good sign. There still exists a little downward slope, however it requires much more time to train and possibly some tuning in hyper parametes.

Though, I think this is a good result for this kind of a model (cross-entropy loss 0.26763), which is much higher than was shown by MLE models.

In [55]:
#Define a dictionary that will convert the RNN output in numbers back to characters 
num_to_char = dict((i, c) for i, c in enumerate(chars))

In [58]:
#Provide our model with random seed character from which it will generate a sequence of characters
start = np.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" веду восточный мордор подзамочный город оживёт водоёмах сточных водах проёмах барочных окон e2 e4 ра "


In [59]:
#Finally, generate rap
for i in range(500):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = num_to_char[index]
    seq_in = [num_to_char[value] for value in pattern]

    sys.stdout.write(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]

здат окну горов полигают оболе косорый год вышно бльнь поинм сорех чёрных пододит здёт роин зиву спор болет коробки свой фоека собода сельные мира стросок поле стали вывали соберик понале миме систу молице босточли бесе собой пол тхих просто прохиран врей послелных залаленнокнго слова полинил дорода полизно тебе лолотик тменти дород всё поле ребе собой пока ного бротить коров страший отсоровах устал оттаться току заслышал некоритовал просто дай нам просто конплекство косора дом стал ваш рэп баш 

To conclude, I choose LSTM RNN model among the presented models, since it has relatively low cross-entropy loss and more lively text generation.

This model seems as a powerful tool to work with texts, but requires much higher resources to train.
To my mind, in order to increase the quality of generated text by this model it is necessary to add much more training data and probably make more complex RNN structure. Since our data is now constrained to 14 tracks, the model sees some special words only ones and never again (e.g. 'хипари', 'дикари'), that is why sometimes it can partly borrow some verses with a little adjustment.

In [ ]:
#Save our loss history in case we construct a graph
hist_file = 'history.json' 
with open(hist_file, mode='w') as f:
    history.to_json(f)